### 下準備

In [9]:
import pandas as pd

#### サンプルとしてtrain_A.tsvを用いる

In [10]:
df = pd.read_csv("data/train/train_A.tsv", sep='\t')
# b_df = pd.read_csv("data/train/train_B.tsv", sep='\t')
# c_df = pd.read_csv("data/train/train_C.tsv", sep='\t')
# d_df = pd.read_csv("data/train/train_D.tsv", sep='\t')

In [11]:
test_df = pd.read_csv("data/test.tsv", sep='\t')
sample_df = pd.read_csv("data/sample_submit.tsv", sep='\t', header=None, names=["user_id", "product_id", "rank"])

# 目的（何をデータから見つければよいのか？）

## 評価関数

>予測精度の評価は、nDCG(normalized discounted cumulative gain)を使用します（右図参照）。この値は、モデルの性能が良いほど大きくなり、1に近くなります。**関連度（relevance）はcv（コンバージョン）を3、cl（広告をクリック）を2、pd（商品ページ閲覧）を1、それ以外は0**とします。ただし**コンバージョンは広告経由のみ評価対象**とします。**クエリごとの最大推薦数kは22**とします。予測値の出力形式についてはダウンロードページの応募用サンプルファイルをご参照ください。また、test.tsvに記載されているすべてのユーザーに対して予測を行ってください。  
引用: https://deepanalytics.jp/compe/45

>![image](https://i.deepanalytics.jp/i/wh0t4i3541)

### 応募用サンプルファイル

In [12]:
sample_df.head(25)

,user_id,product_id,rank
0,0000008_A,00000000_a,0
1,0000008_A,00000001_a,1
2,0000008_A,00000002_a,2
3,0000008_A,00000003_a,3
4,0000008_A,00000004_a,4
5,0000008_A,00000005_a,5
6,0000008_A,00000006_a,6
7,0000008_A,00000007_a,7
8,0000008_A,00000008_a,8
9,0000008_A,00000009_a,9


つまり、**ユーザーごとに商品をランキング付けして上から２２個推薦すれば、nDCGの評価関数を用いて評価値がわかる。**

# どうやって商品をランキング付けすればよいのか？

## 1. 協調フィルタリング

協調フィルタリングとは、**ユーザーとアイテムのマトリックスから、アイテム同士の類似度やユーザー同士の類似度を算出して、その類似度から推薦するアイテムを決定する**手法。  
例えば、ユーザー同士の類似度を用いるとすると、その手順は、  

1. 類似度からユーザーAの好みと類似した好みをもつユーザーBを見つける。  
2. ユーザーBのアイテムに対する評価値からユーザーAがまだ評価していないアイテムの評価値を算出する。  
3. ２で算出した評価値が高い順にユーザーAに対して推薦する。  

といったようになる。  

以上より、**作業の流れ**としては、  

1. 与えられたデータからユーザーのアイテムに対する評価値のマトリックスを作成する。
2. マトリックスから類似度を計算する。
3. 様々なチューニング方法を試してみる。

### 1-1. 与えられたデータからユーザーとアイテムのマトリックスを作成する。

##### 1-1-1. 与えられたデータはどんなデータか？

ユーザー*(user_id)*が商品*(procuct_id)*に対して*event_type*という行動を*time_stamp*時に行ったというデータが与えられている。

In [22]:
df.head()

,user_id,product_id,event_type,ad,time_stamp
0,0000000_A,00009250_a,1,-1,2017-04-08 12:09:04.629
1,0000000_A,00009250_a,1,-1,2017-04-27 12:55:57.783
2,0000000_A,00014068_a,1,-1,2017-04-08 11:57:53.746
3,0000000_A,00001254_a,1,-1,2017-04-08 12:04:26.008
4,0000000_A,00003316_a,1,-1,2017-04-08 12:05:31.326


#### 1-1-2. データ変換はどのようにするのか？

ユーザーが商品に対してとった行動の種類(*event_type*)・回数・時間から評価値を算出する。  
**評価値の算出方法は・・・・今のところ考えていません。**  
以下は仮の変換法。

In [25]:
def make_new_dataframe(filename, params=[1,2,3,4], debug=True):
    ext = filename.split('.')[-1]
    if ext == "csv":
        sep = ','
    elif ext == "tsv":
        sep = '\t'
    else:
        print("please change file extension")
        return
    df = pd.read_csv(filename, sep=sep)

    new_columns = ["user_id"]
    new_columns.extend(list(set(df.product_id)))

    user_ids = set(df.user_id)

    row_data = []
    debug_flag = 0
    for user_id in user_ids:
        per_df = df.loc[df.user_id == user_id]
        per_product_ids = set(per_df.product_id)
        new_row = pd.Series(index=new_columns)
        new_row["user_id"] = user_id

        for per_product_id in per_product_ids:
            ex_df = per_df[per_df["product_id"] == per_product_id]
            n0 = len(ex_df[ex_df["event_type"] == 0].index)
            n1 = len(ex_df[ex_df["event_type"] == 1].index)
            n2 = len(ex_df[ex_df["event_type"] == 2].index)
            n3 = len(ex_df[ex_df["event_type"] == 3].index)
            value = 1*n1 + 2*n2 + 3*n0 + 4*n3
            new_row[per_product_id] = value

        new_row = new_row.fillna(0)
        row_data.append(new_row)
        debug_flag += 1
        if ((debug_flag > 10) and (debug==True)):
            break
    new_df = pd.DataFrame.from_records(row_data)
    
    if debug == True:
        return new_df
    else:
        new_filename = filename.split(".")[0].split("/")[-1] + "_new.csv"
        new_df.to_csv(new_filename)

#### 1-1-3. 変換後のデータはどのようなマトリックスか？

In [26]:
make_new_dataframe("data/train/train_A.tsv")

,user_id,00009727_a,00002499_a,00004721_a,00012027_a,00007324_a,00011825_a,00011651_a,00000486_a,00009244_a,...,00005844_a,00011722_a,00006425_a,00005852_a,00004411_a,00010524_a,00000148_a,00004185_a,00008316_a,00011362_a
0,0021932_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0003878_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0039596_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0033690_A,0,0,0,0,0,0,0,0,0,...,25,0,0,0,0,0,0,0,0,0
4,0000670_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0054160_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0006791_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0055694_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0007790_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0022936_A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1-2. マトリックスから類似度を算出する。

上のマトリックスをみると評価値が0、すなわち**ユーザーがまだ評価していないものが多々含まれている**。  
この状況でそのまま類似度を計算しても、適切な類似度を算出することができない（らしい）。  
ゆえに、**マトリックスの次元を削減し、特徴量を抽出する必要がある**。

#### 1-2-1.マトリックスの次元削減のために Matrix Factorization を行う。

>m人のユーザとn個のアイテムを考えます。  
先ほどの例では、ユーザはn次元のベクトルで表現されることになりますが、これを$$ m>k>0 $$であるk次元に次元削減して変換することを目指します。  
これは評価値を表すm×nの行列Rに対して  
これをユーザ要素を表すk×mの行列Pとk×mの行列Qを考え以下のように近似します  

>$$ R≈P^TQ $$  

>図で表すと以下のような形になります。  

>![image](https://camo.qiitausercontent.com/be358dc385b10bfc0359be215a443f87e3c25f66/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f3232342f323233382f34393430306438342d373261322d656464342d623861622d3634313330316235626462642e706e67)

 詳細は、https://qiita.com/ysekky/items/c81ff24da0390a74fc6c へ  
 以下はそのサンプルコード

In [8]:
import numpy

def get_rating_error(r, p, q):
    return r - numpy.dot(p, q)


def get_error(R, P, Q, beta):
    error = 0.0
    for i in range(len(R)):
        for j in range(len(R[i])):
            if R[i][j] == 0:
                continue
            error += pow(get_rating_error(R[i][j], P[:,i], Q[:,j]), 2)
    error += beta/2.0 * (numpy.linalg.norm(P) + numpy.linalg.norm(Q))
    return error


def matrix_factorization(R, K, steps=5000, alpha=0.0002, beta=0.02, threshold=0.001):
    P = numpy.random.rand(K, len(R))
    Q = numpy.random.rand(K, len(R[0]))
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] == 0:
                    continue
                err = get_rating_error(R[i][j], P[:, i], Q[:, j])
                for k in range(K):
                    P[k][i] += alpha * (2 * err * Q[k][j])
                    Q[k][j] += alpha * (2 * err * P[k][i])
        error = get_error(R, P, Q, beta)
        if error < threshold:
            break
    return P, Q


R = numpy.array([
        [5, 3, 0, 1],
        [4, 0, 0, 1],
        [1, 1, 0, 5],
        [1, 0, 0, 4],
        [0, 1, 5, 4],
        ]
    )
nP, nQ = matrix_factorization(R, 2)
nR = numpy.dot(nP.T, nQ)
nR

array([[ 5.01270433,  2.96556318,  3.99719371,  0.99587408],
       [ 3.98555758,  2.36743834,  3.37482539,  0.99580861],
       [ 1.06233172,  0.85219158,  5.45703507,  4.99221243],
       [ 0.96910713,  0.75098086,  4.43357437,  3.98931305],
       [ 1.70335314,  1.18118297,  4.93283613,  4.04574714]])

>引用: https://qiita.com/ysekky/items/c81ff24da0390a74fc6c 

#### 1-2-2. 類似度の算出

* ピアソン相関係数
* コサイン類似性  
などなどあるようですが、これ以降はまだ調べ切れていません・・・

### モデルベースの協調フィルタリング

今まで説明した協調フィルタリングはメモリベース、つまりデータをそのまま用いて考えていたが、  
クラスタリングや回帰などを使って、モデルベースに変更して協調フィルタリングを行う方法もある(らしい)。  
ここ(http://www.kamishima.net/archive/recsysdoc.pdf, http://d.hatena.ne.jp/EulerDijkstra/20130407/1365349866) に詳しく載ってそう。

参考  
http://d.hatena.ne.jp/EulerDijkstra/20130407/1365349866  
https://takuti.me/note/coursera-recommender-systems/  
https://qiita.com/ysekky/items/c81ff24da0390a74fc6c  
https://www.slideshare.net/KentaOku/ss-50762836  
https://deepanalytics.jp/compe/45  
https://www.slideshare.net/masayuki1986/recommendation-ml

## 2. 内容ベースフィルタリング

**今回は利用することができない**

>推薦対象となるコンテンツの情報とユーザの嗜好データをマッチングし，類似性が高いコンテンツを推薦する手法  
引用：http://gihyo.jp/dev/serial/01/information-recommendation-system/0003?page=2

### 今回利用することができない理由

* 商品のコンテンツに関する情報とユーザーの嗜好に関する情報が一切ないため。  

## 3. バンディットアルゴリズム

限られた試行回数で選択肢を探索し、その中からよりよい結果が出るだろう選択肢を選び出すアルゴリズム。  
ただ、**今回は利用することはできない**と考えられる。

### 今回利用できない理由

* 与えられているデータは、ユーザーが何かしらの行動をしてくれたデータ。  
    * この商品を推薦したけど、ダメだった・・・みたいな情報が存在しない。  
* 新しく探索を行うことができない。
    * まだ何の行動も起こされていない商品について推薦することが不可能。

### 簡単な例

あるユーザーAに対し、 X,Y,Zを配信することを考える。 
* **探索**フェーズ  
    1. Xを配信。→クリック×  
    2. Yを配信。→クリック○！ 
    3. Zを配信。→クリック○！  
    4. Xを配信。→クリック×
    5. Yを配信。→クリック○！  
    6. Zを配信。→クリック×


* **活用**フェーズ  
    * Yのクリックされる確率が一番高いのでは？？？  
    →今後はYを配信する。  

参考  
https://www.slideshare.net/greenmidori83/ss-28443892

## 4. ベイジアンネットワーク

与えられたデータの事象または事象同士の因果関係を考え、それらの確率または条件付確率をノード（節点）として持つ有向の確率グラフで表したもの。 

>簡単に数式的に言うと・・・  
事象$X_1, X_2, \cdots, X_n$の同時確率$P(X_1, X_2, \cdots, X_n)$を  
$$P(X_1, X_2, \cdots, X_n)=P(X_1)P(X_2|X_1)P(X_3|X_1)\cdots$$  
のように、同時確率を確率と条件付確率の積で表せるようにするということ。

ある事象が起きたとき、その確率グラフを用いて次に起こりそうな事象の確率を算出して、起こる可能性の高い事象の推薦を行う。 

今回、**利用はできそう**ではある。

### 今回利用するにあたってのメリット・デメリット（仮説）

#### メリット
* 日付の最初の方のデータを学習に利用し、最終週（最終日でもなんでもよい）のデータを後で与えることで、次の週の確率が予測できそう。  

#### デメリット
* 理解するのも大変そうだし、実装するのも大変そう。
* 確率がデータから推定できないようなユーザー・商品に関しては、確率を予測で与えることしかできず、うまくいかなさそう。

### どのようにしてデータから確率グラフを作成するのか？ 

#### 例：MWSTアルゴリズム

* 仮定  
確率グラフは木構造である

* よりよいベイジアンネットワークとは？  
    * $P(X_1, X_2, \cdots, X_n)$　と　$P(X_1)P(X_2|X_1)P(X_3|X_1)\cdots$　の差が小さいネットワーク  
    →事象間の相互情報量の最大化問題へ

* 手順
    1. 事象間の相互情報量を計算する。
    2. 相互情報量の多い事象のペアからアーク（枝）として追加していく。  
        ただし、ループができたらスキップし、次に相互情報量の多い事象ペアを参照する。
    3. アーク数がn-1になるまで繰り返す。

参考  
https://www.slideshare.net/Erika_Fujita/ss-50118591  
https://www.slideshare.net/hamadakoichi/ss-4552939  
https://staff.aist.go.jp/y.motomura/paper/BSJ0403.pdf  
